In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
from tqdm import tqdm
import re

### NEW 분철행 추가

In [2]:
#### 재작성할 증서목록 파일 ####
wd = r"C:\Users\SL\Desktop"
filename = "통합 문서2.xlsx"
##############################
path_excel = join(wd, filename)
ori_df = pd.read_excel(path_excel, dtype=str)
ori_df

,증서상 매각사,원증서번호,증서유무,채무자키,채무자명,채무자주민번호,정렬번호,전북1차발송,동양1차발송,에이원1차발송,비고,종결,전북,에이원,동양,계좌분리건
0,케이에스,1017,0,20428925,탁정석,480828-1,케이에스1017_00,NaN,0,0,NaN,NaN,NaN,0,0,0
1,케이에스,1018,0,20409091,임미경,650529-2,케이에스1018_00,NaN,0,0,NaN,NaN,NaN,0,0,0
2,케이에스,1019,0,20428927,한만형,750121-1,케이에스1019_00,NaN,0,0,NaN,NaN,NaN,0,0,0
3,케이에스,1020,0,20429835,허민정,760515-2,케이에스1020_00,NaN,0,0,NaN,NaN,NaN,0,0,0
4,케이에스,1021,0,20428920,최영,581215-1,케이에스1021_00,NaN,0,0,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,케이에스,1474,0,20428701,박준식,710216-1,케이에스1474_00,NaN,0,0,NaN,NaN,NaN,0,0,0
458,케이에스,1475,0,20429565,휴림종합건설,212-81-7,케이에스1475_00,NaN,0,0,NaN,NaN,NaN,0,0,0
459,케이에스,1476,0,20429499,동영,401-81-2,케이에스1476_00,NaN,0,0,NaN,0,NaN,0,0,0
460,케이에스,1477,NaN,20428517,김미선,721015-2,케이에스1477_00,NaN,0,0,NaN,1,NaN,0,0,0


In [1]:
# 쉼표, /, () 사용한 셀 있는지 체크하고 수작업하거나, 그에 맞춰 코드 수정

# 문자열 오름차순 정렬 순서 : 숫자형태만 쓰면 상관없으나 기호가 섞이면 문제가 됨.
# 기준1) 숫자문자열은 zfill한 것처럼 정렬됨 >>> 1, 2, 10으로 문제없음. 그러나 언더바나 하이픈 같은 기호 다음은 zfill 적용x >>> 1_1, 1_10, 1_2
# 기준2) 숫자문자열과 기호 : 0, -, _ >>> 따라서 10이 1_1보다 먼저 나와버림
# 결론1) -, _를 쓰건 안 쓰건 분철번호는 자릿수를 맞춰서 해야함 (순번이 필요 없지만 가독성이 안 좋은 단점)
# 결론2) -나 _를 쓰려면 전체에 똑같이 사용해줘야 함. (아래 코드는 이걸 새순번으로 만듦)

### ~있는 거 나누기

In [15]:
물결 = re.compile("~")
하이픈 = re.compile("\-")
result = []
원번호비교 = ""
for i, v in tqdm(ori_df.iterrows(), total=len(ori_df.index)) :
    
    if 물결.search(v.원증서번호) : 
        start = 물결.search(v.원증서번호).start()
        end = 하이픈.search(v.원증서번호).start()
        원번호 = v.원증서번호[:start]
        if 원번호비교  != 원번호 :
            원번호비교 = 원번호
            분철번호 = v.원증서번호[end+1:]
            for x in range(int(분철번호)+1) :
                if x == 0 :
                    result.append(원번호)
                else :
                    result.append(원번호+"-"+str(x))
        else : continue
    else : 
        result.append(v.원증서번호)

100%|██████████| 478/478 [00:00<00:00, 26555.02it/s]

478


In [21]:
rst_df = pd.DataFrame(result, columns=["원증서번호"])

In [23]:
rst_df.to_excel(join(wd,"result.xlsx"))

### -만 있는 거

In [3]:
# 새로운 데이터프레임 생성
df_expanded = pd.DataFrame(columns=ori_df.columns)

# 원증서번호 열의 값이 숫자로 끝나는 경우 확장하고 다른 열의 값을 유지하기
for idx, row in ori_df.iterrows():
    if '-' in row['원증서번호']:
        prefix, count = row['원증서번호'].split('-')
        count = int(count)
        for i in range(0, count + 1):
            new_row = row.copy()
            if i == 0 :
                new_row['원증서번호'] = f"{prefix}"
            else : 
                new_row['원증서번호'] = f"{prefix}-{i}"
            df_expanded = df_expanded.append(new_row, ignore_index=True)
    else:
        df_expanded = df_expanded.append(row, ignore_index=True)

C:\Users\SL\AppData\Local\Temp\ipykernel_41244\1751347812.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_expanded = df_expanded.append(row, ignore_index=True)
C:\Users\SL\AppData\Local\Temp\ipykernel_41244\1751347812.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_expanded = df_expanded.append(new_row, ignore_index=True)


In [4]:
# 재차분리된 것 중복값 제거
df_expanded.drop_duplicates(subset=["원증서번호","채무자키"], inplace=True, ignore_index=True)

In [6]:
# 저장
df_expanded.to_excel(join(wd,"result.xlsx"), index=False)

In [5]:
# 전처리 : 에러 확인
물결 = re.compile("~")
하이픈 = re.compile("\-")
for i, v in tqdm(ori_df.iterrows(), total=len(ori_df.index)) :
    if 물결.search(v.원증서번호) : 
        start = 물결.search(v.원증서번호).start()
        end = 물결.search(v.원증서번호).end()
        앞번호 = v.원증서번호[:start]
        뒷번호 = v.원증서번호[end:]
        # 물결 2개 이상
        if len(물결.findall(v.원증서번호)) > 1 : 
            print("물결 2개 이상", v.원증서번호)
        # 하이픈 2개이상
        if len(하이픈.findall(앞번호))>1 or len(하이픈.findall(뒷번호))>1 :
            print("하이픈 2개 이상", v.원증서번호)
        
    # else : 
    #     if len(하이픈.findall(v.원증서번호)) > 1 : 
    #         start = 물결.search(v.원증서번호).start()
    #         end = 물결.search(v.원증서번호).end()
    #         앞번호 = v.원증서번호[:start]
    #         뒷번호 = v.원증서번호[end:]
    #         # 하이픈 2개이상
    #         if len(하이픈.findall(앞번호))>1 or len(하이픈.findall(뒷번호))>1 :
    #             print("하이픈 2개 이상", v.원증서번호)
                
# (부연설명) 지우기 : 여기서 지우면 부연설명을 더는 볼수가 없네..
# ori_df["원증서번호"] = ori_df["원증서번호"].apply(lambda x : 부연설명.sub("", x))

100%|██████████| 2871/2871 [00:00<00:00, 37276.53it/s]


In [7]:
정렬용순번, 매각사, 원장번호, 새순번, 새증서번호, 채무자키, 비고 = [],[],[],[],[], [], []
물결 = re.compile("~")
하이픈 = re.compile("-")
부연설명 = re.compile("\(.*\)")
for i, v in tqdm(ori_df.iterrows(), total=len(ori_df.index)) :
    try : 
        원래원장번호 = 부연설명.sub("", v.원장번호)    
        
        # 물결 있으면
        if 물결.search(원래원장번호) : 
            start = 물결.search(원래원장번호).start()
            end = 물결.search(원래원장번호).end()
            앞번호 = 원래원장번호[:start]
            뒷번호 = 원래원장번호[end:]
            
            # 앞번호
            # 앞번호에 하이픈 있으면
            if 하이픈.search(앞번호) :
                # 4자리 증서번호 만들기
                앞증서번호 = 앞번호[ : 하이픈.search(앞번호).start()]
                # 2자리 분철번호 만들기 (int)
                앞분철번호 = 앞번호[하이픈.search(앞번호).end() : ]
            # 앞번호에 하이픈 없으면
            else : 
                앞증서번호 = 앞번호
                앞분철번호 = "0"
            
            # 뒷번호
            if 하이픈.search(뒷번호) :
                뒷증서번호 = 뒷번호[ : 하이픈.search(뒷번호).start()]
                뒷분철번호 = 뒷번호[하이픈.search(뒷번호).end() : ]
            else : 
                뒷증서번호 = 뒷번호
                뒷분철번호 = "0"

            # 증서번호 같은가?
            if 앞증서번호 == 뒷증서번호 : 
                증서번호 = 앞증서번호
                # 분철번호만큼 반복해서 값 채우기
                for i in range(int(앞분철번호), int(뒷분철번호)+1) : ############값채우기
                    분철번호 = str(i)
                    정렬용순번.append(v.정렬용순번)
                    매각사.append(v.매각사)
                    원장번호.append(v.원장번호)
                    새순번.append(증서번호.zfill(4) + "_" + 분철번호.zfill(2))
                    새증서번호.append(증서번호 if 분철번호 == "0" else 증서번호 + "_" + 분철번호)
                    채무자키.append(v.채무자키)
                    비고.append("")
            # error : 물결있는데 증서번호가 다른 경우
            else : 
                raise Exception("앞증서번호와 뒷증서번호가 다름")
        
        # 물결 없으면
        else :
            # 하이픈 있으면
            if 하이픈.search(원래원장번호) : 
                증서번호 = 원래원장번호[ : 하이픈.search(원래원장번호).start()]
                분철번호 = 원래원장번호[하이픈.search(원래원장번호).end():]
            # 하이픈 없으면
            else : 
                증서번호 = 원래원장번호
                분철번호 = "0"

            정렬용순번.append(v.정렬용순번) ############값채우기
            매각사.append(v.매각사)
            원장번호.append(v.원장번호)
            새순번.append(증서번호.zfill(4) + "_" + 분철번호.zfill(2))
            새증서번호.append(증서번호 if 분철번호 == "0" else 증서번호 + "_" + 분철번호)
            채무자키.append(v.채무자키)
            비고.append("")
    
    except Exception as e :
        정렬용순번.append(v.정렬용순번)
        매각사.append(v.정렬용순번[:하이픈.search(v.정렬용순번).start()])
        원장번호.append(원래원장번호)
        새순번.append("") ###
        새증서번호.append("") ###
        채무자키.append(v.채무자키)
        비고.append(e.args[0])
        continue

100%|██████████| 2871/2871 [00:00<00:00, 25406.08it/s]


In [9]:
# 결과 저장하기
result_df = pd.DataFrame({
    "정렬용순번":정렬용순번,
    "매각사":매각사,
    "원장번호":원장번호,
    "새순번":새순번,
    "새증서번호":새증서번호,
    "채무자키":채무자키,
    "비고":비고
})
result_df.to_excel(join(wd, "result_list.xlsx"), index=False)

# 분철행 추가하기

In [11]:
#### 재작성할 증서목록 파일 ####
wd = r"D:\3.자산\프로젝트\2023 실물증서정리\광주"
filename = "0.기타 매각사 증서번호.xlsx"
##############################
path_excel = join(wd, filename)
ori_df = pd.read_excel(path_excel, sheet_name="리스트", dtype=str).fillna("")
# 정렬번호로 컬럼명 바꾸기
# ori_df.rename(columns={"정렬(not unique)":"정렬번호"}, inplace=True)
# 정렬
ori_df.sort_values("정렬번호", inplace=True)
ori_df

,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건
1816,,산은,,62,0062_00,산은0062_00,62,,,,,,,
1895,,산은,,66,0066_00,산은0066_00,66,,,,,,,
1858,,산은,,84,0084_00,산은0084_00,84,,,,,,,
1897,,산은,,92,0092_00,산은0092_00,92,,,,,,,
1898,,산은,,110,0110_00,산은0110_00,110,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,,케이에스,,910~910-1,0910_01,케이에스910~910_01,910_1,,,,,,,
1811,,케이에스,,913~913-1,0913_00,케이에스913~913_01,913,,,,,,,
1812,,케이에스,,913~913-1,0913_01,케이에스913~913_01,913_1,,,,,,,
1813,,케이에스,,940~940-1,0940_00,케이에스940~940_01,940,,,,,,,


In [12]:
# 대표번호, 분철번호열 추가하기
ori_df["대표번호"] = ori_df.정렬번호.str.split('_').str[0]
ori_df["분철번호"] = ori_df.정렬번호.str.split('_').str[1]
ori_df.head()

,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건,대표번호,분철번호
1816,,산은,,62,0062_00,산은0062_00,62,,,,,,,,산은0062,00
1895,,산은,,66,0066_00,산은0066_00,66,,,,,,,,산은0066,00
1858,,산은,,84,0084_00,산은0084_00,84,,,,,,,,산은0084,00
1897,,산은,,92,0092_00,산은0092_00,92,,,,,,,,산은0092,00
1898,,산은,,110,0110_00,산은0110_00,110,,,,,,,,산은0110,00


In [13]:
# 대표번호로 그룹화하기
grouped_df = ori_df.groupby("대표번호")

In [14]:
ori_df.columns

Index(['구정렬용순번', '증서상 매각사', '실제매각사', '원증서번호', '정렬용', '정렬번호', '새증서번호', '채무자키',
       '채무자명', '채무자주민번호', '에러', '에이원', '동양', '계좌분리건', '대표번호', '분철번호'],
      dtype='object')

In [15]:
result_df = pd.DataFrame(None, columns=ori_df.columns)
for g_name, g_data in tqdm(grouped_df, total=len(grouped_df)):
    result_df = pd.concat([result_df, g_data])
    분철번호 = g_data["분철번호"].astype(int)
    최소값 = 분철번호.min()
    최대값 = 분철번호.max()

    # 연속되지 않는 분철번호 찾기
    missing = [num for num in range(최소값, 최대값+1) if num not in 분철번호.values]

    if missing :
        temp_df = pd.DataFrame([{}]*len(missing), columns=ori_df.columns)
        temp_df["분철번호"] = [str(i).zfill(2) for i in missing]
        temp_df["정렬용"] = [g_data["정렬용"].values[0][:-2] + x for x in temp_df["분철번호"].values]
        temp_df["정렬번호"] = [g_data["정렬번호"].values[0][:-2] + x for x in temp_df["분철번호"].values]
        temp_df["대표번호"] = g_data["정렬번호"].values[0][:-3]
        temp_df[["구정렬용순번", "원증서번호", "실제매각사", "원증서번호", "새증서번호", "에러", "에이원", "동양", "계좌분리건"]]= ""
        temp_df[["증서상 매각사", "채무자키", "채무자명", "채무자주민번호"]] = g_data.loc[g_data.index[0],["증서상 매각사", "채무자키", "채무자명", "채무자주민번호"]]
        result_df = pd.concat([result_df, temp_df])

result_df

  0%|          | 0/2648 [00:00<?, ?it/s]

100%|██████████| 2648/2648 [00:01<00:00, 2638.36it/s]


,구정렬용순번,증서상 매각사,실제매각사,원증서번호,정렬용,정렬번호,새증서번호,채무자키,채무자명,채무자주민번호,에러,에이원,동양,계좌분리건,대표번호,분철번호
1816,,산은,,62,0062_00,산은0062_00,62,,,,,,,,산은0062,00
1895,,산은,,66,0066_00,산은0066_00,66,,,,,,,,산은0066,00
1858,,산은,,84,0084_00,산은0084_00,84,,,,,,,,산은0084,00
1897,,산은,,92,0092_00,산은0092_00,92,,,,,,,,산은0092,00
1898,,산은,,110,0110_00,산은0110_00,110,,,,,,,,산은0110,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,,케이에스,,910~910-1,0910_01,케이에스910~910_01,910_1,,,,,,,,케이에스910~910,01
1811,,케이에스,,913~913-1,0913_00,케이에스913~913_01,913,,,,,,,,케이에스913~913,01
1812,,케이에스,,913~913-1,0913_01,케이에스913~913_01,913_1,,,,,,,,케이에스913~913,01
1813,,케이에스,,940~940-1,0940_00,케이에스940~940_01,940,,,,,,,,케이에스940~940,01


In [16]:
result_df.to_excel(join(wd, "분철번호채우기.xlsx"), index=False)

In [43]:
# 그룹화된 DataFrame을 리스트로 저장
grouped_dataframes = [g_data for g_name, g_data in grouped_df]

# 그룹화된 데이터프레임을 일반 데이터프레임으로 변환
merged_df = pd.concat(grouped_dataframes)

# 데이터프레임을 엑셀 파일로 저장
merged_df.to_excel('xxx.xlsx', index=False)

### 채무자키 찾기

In [4]:
excel_path = r"D:\3.자산\프로젝트\2023 실물증서정리\광주\0.광주 증서목록_분철번호채우기.xlsx"
df = pd.read_excel(excel_path, dtype=str, sheet_name='번호부여').fillna("")
df

,증서상 매각사,원증서번호,증서유무,채무자키,채무자명,채무자주민번호
0,,,,,김종철,540903
1,,,,,신범철,690711
2,,,,,유현삼,740325
3,,,,,고혜련,790726
4,,,,,양성열,700118
...,...,...,...,...,...,...
236,,,,1803000020,,
237,,,,1803000047,,
238,,,,1803000101,,
239,,,,1803000007,,


In [5]:
path_pkl_dir = r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회"
debt_pkl = "채무자조회새창_20221220_1128"
df_debt = pd.read_pickle(join(path_pkl_dir, debt_pkl+".pkl")) #성명, 주민번호인, 매각사구분, 매각사, 원채권사 (아래서도 공통) # 현재원금 순으로 정렬됨
df_debt

,채무자키,매각사구분,채권구분,성명,주민번호인,담당자,채무상태,성별,연령,원채권사,매각사,최초원금,최초합계,현재원금,관리자기타,비고,종결일,보증인성명,보증인상태,타채무자키
50942,20423144,BNK-01,NPL,이환복(덕양건기),460801-1038015,지영숙,정상,남자,76,BNK캐피탈,BNK캐피탈,31515689,35825637,31515689,9001000669,20111031,,,,9001000669
30651,20422983,BNK-01,NPL,서정택(김양비어),471108-1157017,진인철,정상,남자,75,BNK캐피탈,BNK캐피탈,2418974,2710764,2418974,9001000444,20190920,,,,9001000444
30650,20423372,BNK-01,NPL,서정태,480219-1767923,양진욱,파산(면책),남자,74,BNK캐피탈,BNK캐피탈,23771451,28185225,23761451,9001000997,,,,,9001000997
55830,20423423,BNK-01,NPL,정상갑(금성종합티브이),520220-1093228,정용두,정상,남자,70,BNK캐피탈,BNK캐피탈,4062926,7679204,4062926,9001001076,20140314,,,,9001001076
40782,20423411,BNK-01,NPL,윤재술(도서출판예은),520707-1447318,신용회복,신용회복(확정),남자,70,BNK캐피탈,BNK캐피탈,7252284,9361406,6526783,9001001062,주소변동무.,,,,9001001062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43306,20407837,흥국화재-01,NPL,이문지,890220-2151647,이경옥,정상,여자,33,흥국화재,흥국화재해상보험㈜,4441673,5091870,1550157,9003000842,,,,,9003000842
673,20423828,흥국화재-01,NPL,강명진,890607-1590416,김미정,정상,남자,33,흥국화재,흥국화재해상보험㈜,13200000,15743483,13200000,9003000435,,,,,9003000435
27492,20455107,흥국화재-01,NPL,박철혁,900121-1851911,종결,종결,남자,32,흥국화재,흥국화재해상보험㈜,7500007,10701775,0,9003000436,,2021-05-11,,,9003000436
6842,20411780,흥국화재-01,NPL,김동수,901021-1836911,신용회복,신용회복(확정),남자,32,흥국화재,흥국화재해상보험㈜,4333336,5157192,4153386,9003000492,2020-12-31 거주불명자-말소,,,,9003000492


In [7]:
df_debt[df_debt.관리자기타=="1601000193"]

,채무자키,매각사구분,채권구분,성명,주민번호인,담당자,채무상태,성별,연령,원채권사,매각사,최초원금,최초합계,현재원금,관리자기타,비고,종결일,보증인성명,보증인상태,타채무자키
69334,20407942,한울가람,NPL,홍연순,720521-2240714,홍정기,정상,여자,50,테크메이트,한울가람대부㈜,2965136,6732206,2763406,1601000193,의정부16개회53614/17.9.29.폐지(인가후),,신진화,개인회생(확정),1601000193


In [11]:
result = []
for i, v in df.iterrows() :
    keys = None
    if v.채무자키 == "" :
        keys = df_debt[(df_debt.성명.str.contains(v.채무자명)) & (df_debt.주민번호인.str.match(v.채무자주민번호))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    else :
        keys = df_debt[df_debt.관리자기타 == v.채무자키].채무자키.values
    
    keys_set = set(keys)
    
    result.append(keys_set.values)

df["채무자키"] = result
df.to_excel("xxx.xlsx", index=False)

AttributeError: 'set' object has no attribute 'values'